In [1]:
import fff
from hexdump import hexdump as hd

disk = fff.DiskImage('/Users/xin/practice1.dd.zip')

disk.volume

  #  Slot      Start  End       Length  Description
---  ------  -------  ------  --------  ----------------------------------
  1  0             0  0              1  Master Boot Record
  2  0:-           1  31            30  Unallocated
  3  0:0          32  30719      30688  Linux native partition (0x83)
  4  0:1       30720  98303      67584  Linux swap (0x82)
  5  0:2       98304  208895    110592  Linux native partition (0x83)
  6  0:-      208896  212991      4095  Unallocated
  7  0:3      212992  417791    204800  DOS 3.3+ Extended Partition (0x5)
  8  1        212992  212992         1  Extended Boot Record
  9  1:-      212993  213023        30  Unallocated
 10  1:0      213024  286719     73696  Linux native partition (0x83)
 11  1:1      286720  417791    131072  DOS 3.3+ Extended Partition (0x5)
 12  2        286720  286720         1  Extended Boot Record
 13  2:-      286721  286751        30  Unallocated
 14  2:0      286752  417791    131040  Linux native partition (0x83

In [2]:
print(disk.volume[4])

  #  Slot      Start    End    Length  Description        CHS
---  ------  -------  -----  --------  -----------------  ---------------------
  4  0:1       30720  98303     67584  Linux swap (0x82)  CHS 15/0/1 - 47/63/32


In [3]:
fat = disk.volume[10].filesystem
print(fat.boot_sector)

In [4]:
print(disk.volume[3])
hd(disk.volume[14].sectors[0][0:0x20])

  #  Slot      Start    End    Length  Description                    CHS
---  ------  -------  -----  --------  -----------------------------  --------------------
  3  0:0          32  30719     30688  Linux native partition (0x83)  CHS 0/1/1 - 14/63/32
00000000: EB 52 90 4E 54 46 53 20  20 20 20 00 02 02 00 00  .R.NTFS    .....
00000010: 00 00 00 00 00 F8 00 00  20 00 40 00 20 60 04 00  ........ .@. `..


In [5]:
ntfs = disk.volume[14].filesystem
ntfs.boot_sector

Field                            Value
-------------------------------  --------------------------
JMP                              eb5290
Signature                        NTFS    (4e54465320202020)
Bytes Per Sector                 512
Sectors Per Cluster              2
...                              ...
Media Descriptor                 248
Sectors Per Track                32
Number of Heads                  64
Hidden Sectors                   286752
...                              ...
Total Sectors                    131039
$MFT Cluster Number              16
$MFTMirr Cluster Number          32759
Cluster Per File Record Segment  1
...                              ...
Cluster Per Index Buffer         4
...                              ...
Volume Serial Number             7829755030752268642
...                              ...
Bootstrap Code                   fa33c08ed0bc007cfb68
Marker                           55aa

In [6]:
print(ntfs.mft.mft_entry)
[print(a) for a in ntfs.mft.mft_entry.attrs]

320 0x12 []
392 0x11 []
Field                          Value
-----------------------------  --------------
Signature                      FILE(46494c45)
Offset to Fixup                48
Entry Count in Fixup Array     3
$LogFile Sequence Number       0
Sequence Number                1
Link Count                     1
Attribute Offset               56
Flags                          0b1
Used Size of MFT Entry         408
Allocated Size of MFT Entry    1024
File Reference to Base Record  0
Next Attribute ID              4
#attributes                    4
Field                   Value
----------------------  -------------------------
Type ID                 16 ($StandardInformation)
Name                    b''
Size                    96
Non-Resident Flag       False (Resident)
Name Length             0
Name Offset             24
Flags                   0b0
Attribute ID            0
Attribute Length        3577643008
Created                 116444736000000000
Modified                1164447

[None, None, None, None]

In [7]:
print(ntfs.mft.attr(type_id='$DATA'))

Field                  Value
---------------------  -------------------
Type ID                128 ($DATA)
Name                   b''
Size                   72
Non-Resident Flag      True (Non-Resident)
Name Length            0
Name Offset            64
Flags                  0b0
Attribute ID           1
Attribute Length       268499730
Starting VCN           0
Last VCN               250
DataRuns Offset        64
Compression Unit Size  0
Allocated Size         257024
Actual Size            257024
Compressed Size        257024
#Data Runs             1
Data                   None


In [8]:
print(ntfs.mft.attr(type_id='$DATA').data_runs)
hd(ntfs.mft.read2(count=1024))

[(251, 16)]
00000000: 46 49 4C 45 30 00 03 00  00 00 00 00 00 00 00 00  FILE0...........
00000010: 01 00 01 00 38 00 01 00  98 01 00 00 00 04 00 00  ....8...........
00000020: 00 00 00 00 00 00 00 00  04 00 00 00 00 00 00 00  ................
00000030: B8 00 00 00 00 00 00 00  10 00 00 00 60 00 00 00  ............`...
00000040: 00 00 18 00 00 00 00 00  48 00 00 00 18 00 00 00  ........H.......
00000050: 00 80 3E D5 DE B1 9D 01  00 80 3E D5 DE B1 9D 01  ..>.......>.....
00000060: 00 80 3E D5 DE B1 9D 01  00 80 3E D5 DE B1 9D 01  ..>.......>.....
00000070: 06 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000080: 00 00 00 00 00 01 00 00  00 00 00 00 00 00 00 00  ................
00000090: 00 00 00 00 00 00 00 00  30 00 00 00 68 00 00 00  ........0...h...
000000A0: 00 00 18 00 00 00 02 00  4A 00 00 00 18 00 01 00  ........J.......
000000B0: 05 00 00 00 00 00 05 00  00 87 05 6B 62 94 C9 01  ...........kb...
000000C0: 00 87 05 6B 62 94 C9 01  00 87 05 6B 62 94 C9 01  ...k